<div align="center"><font size=6 color="red">Spam </font><font size=6 color="green"> & Ham</font><font size=6 color="black"> Detection</font></div>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

#impore re for regular expression
import re

# import punctuations
from string import punctuation

#importing stopwords
from nltk.corpus import stopwords

#import the tokenizer
from nltk.tokenize import RegexpTokenizer

#importing the Stemmer
from nltk.stem import PorterStemmer,SnowballStemmer

#importing the Lemmatizer
from nltk.stem import WordNetLemmatizer

# import wordcloud for text visualization
from wordcloud import WordCloud

In [ ]:
import chardet
with open("../input/sms-spam-collection-dataset/spam.csv" , "rb") as f:
    enc = chardet.detect(f.read(1000000))
    
print(enc)

In [ ]:
data = pd.read_csv("../input/sms-spam-collection-dataset/spam.csv" , encoding="Windows-1252")
data.head()

In [ ]:
print(f"Shape of Data: {data.shape}")

In [ ]:
data.isna().sum()

Let's take a look at the colums with lot's of missing values

In [ ]:
# P.S: The display keyword only works in the jupyter notebook enviroment and would raise an error in an actual python file
display(data[data["Unnamed: 2"].notnull()].head())
display(data[data["Unnamed: 3"].notnull()].head())
display(data[data["Unnamed: 4"].notnull()].head())

Text in the "v2" column and the other colums are all of the same text, this could have been caused by commas(,) in the text since we're reading as csv(comma seperated values), we can just add all the columns back together to make a single text

In [ ]:
#for every line where feature "Unnamed: 2" is not_null, we add Feature "Unnamed: 2" to "Text" feature
data["v2"][data["Unnamed: 2"].notna()] = data["v2"][data["Unnamed: 2"].notna()] + data["Unnamed: 2"][data["Unnamed: 2"].notna()]
#for every line where feature "Unnamed: 3" is not_null, we add Feature "Unnamed: 3" to "Text" feature
data["v2"][data["Unnamed: 3"].notna()] = data["v2"][data["Unnamed: 3"].notna()] + data["Unnamed: 3"][data["Unnamed: 3"].notna()]
#for every line where feature "Unnamed: 4" is not_null, we add Feature "Unnamed: 4" to "Text" feature
data["v2"][data["Unnamed: 4"].notna()] = data["v2"][data["Unnamed: 4"].notna()] + data["Unnamed: 4"][data["Unnamed: 4"].notna()]
#drop feature "A","B", and "C"
data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"] , axis=1, inplace=True)

In [ ]:
data.columns = ["label","text"]#rename columns
data.head()

The problem of missing value has been solved with that

### Exploratory Data Analysis

In [ ]:
data["label"].value_counts().plot(kind="bar")
plt.show()

There is a very HUGE imbalance in our dataset

#### Let's take a look at the distribution in our Dataset

In [ ]:
fig , ax = plt.subplots(nrows=6 , ncols=1,figsize=(10,50))
def word_length_count(text):
    return len(text.split())
data["word_length"] = data["text"].apply(word_length_count)
sns.kdeplot(data["word_length"][data["label"] == "ham"] , shade=True , label="ham" , ax = ax[0])
sns.kdeplot(data["word_length"][data["label"] == "spam"] , shade=True , label="spam" , ax=ax[0])
ax[0].set_title("Distribution of Words Count", fontsize=14)
ax[0].set_xlabel("Number of Words in Text" , fontsize=14)
ax[0].set_ylabel("Frequency Rate" , fontsize=14)

def char_length_count(text):
    return len(text)
data["characters_count"] = data["text"].apply(char_length_count)
sns.kdeplot(data["characters_count"][data["label"] == "ham"] , shade=True , label="ham" , ax = ax[1])
sns.kdeplot(data["characters_count"][data["label"] == "spam"] , shade=True , label="spam" , ax=ax[1])
ax[1].set_title("Distribution of Characters Count", fontsize=14)
ax[1].set_xlabel("Number of Characters in Text" , fontsize=14)
ax[1].set_ylabel("Frequency Rate" , fontsize=14)

def punct_count(text):
    return len([char for char in text if char in punctuation])
data["punctuation_count"] = data["text"].apply(punct_count)
sns.kdeplot(data["punctuation_count"][data["label"] == "ham"] , shade=True , label="ham" , ax = ax[2])
sns.kdeplot(data["punctuation_count"][data["label"] == "spam"] , shade=True , label="spam" ,ax = ax[2])
ax[2].set_title("Distribution of Punctuations Count", fontsize=14)
ax[2].set_xlabel("Number of Punctuations in Text" , fontsize=14)
ax[2].set_ylabel("Frequency Rate" , fontsize=14)

def numbers_count(text):
    return len([char for char in text if char.isnumeric()])
data["numbers_count"] = data["text"].apply(numbers_count)
sns.kdeplot(data["numbers_count"][data["label"] == "ham"] , shade=True , label="ham" , ax = ax[3])
sns.kdeplot(data["numbers_count"][data["label"] == "spam"] , shade=True , label="spam",ax = ax[3])
ax[3].set_title("Distribution of Numerical Characters Count", fontsize=14)
ax[3].set_xlabel("Number of Numerical String in Text" , fontsize=14)
ax[3].set_ylabel("Frequency Rate" , fontsize=14)

def stopwords_counts(text):
    return len([word for word in text.split() if word in stopwords.words("english")])
data["stopwords_count"] = data["text"].apply(stopwords_counts)
sns.kdeplot(data["stopwords_count"][data["label"] == "ham"] , shade=True , label="ham" , ax = ax[4])
sns.kdeplot(data["stopwords_count"][data["label"] == "spam"] , shade=True , label="spam" , ax = ax[4])
ax[4].set_title("Distribution of Stopwords Count", fontsize=14)
ax[4].set_xlabel("Number of Stopwords in Text" , fontsize=14)
ax[4].set_ylabel("Frequency Rate" , fontsize=14)

def non_stopwords_counts(text):
    return len([word for word in text.split() if word not in stopwords.words("english")])
data["non_stopwords_count"] = data["text"].apply(non_stopwords_counts)
sns.kdeplot(data["non_stopwords_count"][data["label"] == "ham"] , shade=True , label="ham" , ax = ax[5])
sns.kdeplot(data["non_stopwords_count"][data["label"] == "spam"] ,  shade=True , label="spam" , ax = ax[5])
ax[5].set_title("Distibution of Non-Stopwords Count", fontsize=14)
ax[5].set_xlabel("Number of Non-Stopwords in Text" , fontsize=14)
ax[5].set_ylabel("Frequency Rate" , fontsize=14)
plt.show()

**OBSERVATIONS:**
1. Spam Text has alot of numbers in them while Ham Text has comaparably low numbers in them, this is very logical
2. Spam Text has an average of more words than Ham Text, this is also very logical since scammers with try to convince you and convey extra amount of information to intice unsuspecting people

In [ ]:
for _ in data["text"]:
    print(_)

### Data Preprocessing

In [ ]:
#change text to lowercase
data["text"] = data["text"].str.lower()

#### Let's check is our text containg web links

In [ ]:
display(data[data["text"].str.contains("http")].head())
display(data[data["text"].str.contains("www")].head())

If we look well, we'll see there are some datapoints with "https" | "http" and some with "www", we'll have to deal with this

In [ ]:
# write a function to remove web links
def remove_html(text):
    text = re.sub(r"http://.+\.com" , "" , text)
    text = re.sub(r"http://\S+" , "" , text)
    text = re.sub(r"http://[A-Za-z0-9./?= *&:-]+" , "" , text)
    text = re.sub(r"http.+\.com" , "" , text)
    text = re.sub(r"http//[A-Za-z0-9./?= *&:-]+" , "" ,text)
    text = re.sub(r"www[.A-Za-z0-9/+-]+" , "" , text)
    return text
data["text"] = data["text"].apply(remove_html)

In [ ]:
#Remove numbers and words which are joined with numbers such that the number removal makes them useless
def remove_numbers(text):
    text = re.sub(r"[å£A-Za-z0-9]*\d+[A-Za-z0-9]*" , "" , text)
    return text
data["text"] = data["text"].apply(remove_numbers)

There are alot of instances with text like "å£". For example:
"winner!! as a valued network customer you have been selected to receivea å£900 prize reward! to claim call. claim code. valid hours only"

**Let's deal with that**

In [ ]:
# to view some datapoints with charcters such as "å"
data[data["text"].str.contains("å")].sample(5)

In [ ]:
def remove_fancy_text(word):
    if word.isascii() == False:
        return ""
    else:
        return word
data["text"] = data["text"].apply(lambda x: " ".join([remove_fancy_text(word) for word in x.split()]))

In [ ]:
for _ in data["text"]:
    print(_)

#### Let's get rid of email addresses

In [ ]:
data[data["text"].str.contains("@")].head()

In [ ]:
def remove_mails(text):
    text = re.sub(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+(\.[a-zA-Z]{2,5})" , "" , text)
    return text
data["text"] = data["text"].apply(remove_mails)

In [ ]:
# view amils with "@" in them 
data[data["text"].str.contains("@")]

After cleaning, we still have some text with "@" in them, we'll deal with this in punctattion removel

### Tokenization

In [ ]:
tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S=+')
data["text"] = data["text"].apply(lambda x : tokenizer.tokenize(x))

### Stopwords Removal

In [ ]:
def remove_stopwords(text):
    return [word for word in text if word not in stopwords.words("english")]
data["text"] = data["text"].apply(remove_stopwords)

### Remove Punctautions

In [ ]:
def remove_puncts(text):
    return [word for word in text if word not in punctuation]
data["text"] = data["text"].apply(remove_puncts)

In [ ]:
data["text_length"] = data["text"].apply(lambda x: len(x))

In [ ]:
data.head()

"text_length" column should be less than or equal to "non_stopwords_count" and also similiar to it and if not so it should should only be greater than "non_stopwords_counts" by 1

In [ ]:
data[data["text_length"] > data["non_stopwords_count"]]

### Remove standalone text

removing standalone words like "k","a","m" and the likes

In [ ]:
data["text"] = data["text"].apply(lambda x: [word for word in x if len(word) >= 2])

### Lemmatization

In [ ]:
lemma = WordNetLemmatizer()
data["text"] = data["text"].apply(lambda x: " ".join([lemma.lemmatize(word) for word in x]))

### Stemming

In [ ]:
# stemmer = SnowballStemmer(language="english")
# data["text"].apply(lambda x: " ".join([stemmer.stem(word) for word in x]))

In [ ]:
spam_corpus = ""
for text in data[data["label"] == "spam"]["text"]:
    spam_corpus += " " + text
ham_corpus = ""
for text in data[data["label"] == "ham"]["text"]:
    ham_corpus += " " + text

In [ ]:
spam_wordcloud = WordCloud(background_color="white").generate(spam_corpus)
ham_wordcloud = WordCloud(background_color="white").generate(ham_corpus)
fig,ax= plt.subplots(nrows=1,ncols=2,figsize=(20,100))
ax[0].imshow(spam_wordcloud)
ax[0].set_title("Spam Text" , fontsize=20)
ax[0].axis(False)
ax[0].grid(False)
plt.savefig("Spam Text")
ax[1].imshow(ham_wordcloud)
ax[1].set_title("Ham Text" , fontsize=20)
ax[1].axis(False)
ax[1].grid(False)
plt.show()